# **Cat Image Generator Model**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf

In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Reshape, Flatten, Input, LeakyReLU, BatchNormalization, Conv2D, Conv2DTranspose, Dropout

from tensorflow.keras.optimizers import Adam


In [3]:
# Tentukan direktori data
train_dir = '/content/drive/MyDrive/Colab Notebooks/Capstone/data/train_data'
test_dir = '/content/drive/MyDrive/Colab Notebooks/Capstone/data/test_data'

# Fungsi untuk memuat data
def load_data(directory, target_size=(224, 224)):
    X = []
    y = []
    class_indices = {'kucing': 0}  # Buat pemetaan kelas hanya 'kucing'

    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        img = load_img(img_path, target_size=target_size)
        img_array = img_to_array(img)
        X.append(img_array)
        y.append(class_indices['kucing'])  # Berikan label kelas 'kucing' untuk semua gambar

    X = np.array(X)
    y = np.array(y)
    return X, y, class_indices

# Memuat data latih
X_train, y_train, train_class_indices = load_data(train_dir)

# Memuat data validasi
X_test, y_test, test_class_indices = load_data(test_dir)

# Mendapatkan informasi kelas
num_classes = len(train_class_indices)
print(f"Jumlah kelas: {num_classes}")
print("Pemetaan kelas:", train_class_indices)

Jumlah kelas: 1
Pemetaan kelas: {'kucing': 0}


In [ ]:
# Tentukan hyperparameter
latent_dim = 100
img_shape = (64, 64, 3)
batch_size = 64
epochs = 10000

# Buat model generator
generator = Sequential()
# Mulai dari 4x4 dan upscale hingga 64x64
generator.add(Dense(4 * 4 * 512, input_dim=latent_dim))
generator.add(Reshape((4, 4, 512)))
generator.add(BatchNormalization())
generator.add(LeakyReLU(alpha=0.2))

# 4x4 -> 8x8
generator.add(Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same'))
generator.add(BatchNormalization())
generator.add(LeakyReLU(alpha=0.2))

# 8x8 -> 16x16
generator.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
generator.add(BatchNormalization())
generator.add(LeakyReLU(alpha=0.2))

# 16x16 -> 32x32
generator.add(Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same'))
generator.add(BatchNormalization())
generator.add(LeakyReLU(alpha=0.2))

# 32x32 -> 64x64
generator.add(Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same', activation='tanh'))

# Buat model discriminator
discriminator = Sequential()
discriminator.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same', input_shape=img_shape))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.4))

discriminator.add(Conv2D(128, (4, 4), strides=(2, 2), padding='same'))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.4))

discriminator.add(Conv2D(256, (4, 4), strides=(2, 2), padding='same'))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.4))

discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))

# Kompilasi model
discriminator.compile(loss='binary_crossentropy',
                     optimizer=Adam(learning_rate=0.0002, beta_1=0.5),
                     metrics=['accuracy'])

# Set discriminator ke non-trainable untuk training GAN
discriminator.trainable = False

# Buat model GAN
gan_input = Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy',
           optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

# Loop training
for epoch in range(epochs):
    # Latih discriminator
    real_images = X_train[np.random.choice(X_train.shape[0], batch_size)]
    real_images = tf.image.resize(real_images, (64, 64))
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_images = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Latih generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

    # Tampilkan progress
    print('Epoch ', epoch+1, '/', epochs, ' D-Loss : ', d_loss[0], ' G-Loss : ', g_loss)

# 1. Simpan model generator yang sudah dilatih
generator.save('generator_model.h5')

Output streaming akan dipotong hingga 5000 baris terakhir.
2/2 [==============================] - 0s 32ms/step
Epoch  7366 / 10000  D-Loss :  7.971694415687125e-09  G-Loss :  18.268253326416016
2/2 [==============================] - 0s 30ms/step
Epoch  7367 / 10000  D-Loss :  9.33089250310104e-09  G-Loss :  18.178890228271484
2/2 [==============================] - 0s 31ms/step
Epoch  7368 / 10000  D-Loss :  8.477640811577203e-09  G-Loss :  18.199949264526367
2/2 [==============================] - 0s 30ms/step
Epoch  7369 / 10000  D-Loss :  7.895438081106931e-09  G-Loss :  18.123828887939453
2/2 [==============================] - 0s 38ms/step
Epoch  7370 / 10000  D-Loss :  8.896108738554176e-09  G-Loss :  18.259536743164062
2/2 [==============================] - 0s 31ms/step
Epoch  7371 / 10000  D-Loss :  9.780185550312126e-09  G-Loss :  18.18628692626953
2/2 [==============================] - 0s 29ms/step
Epoch  7372 / 10000  D-Loss :  8.792620853625976e-09  G-Loss :  18.30968093872070

In [5]:


### Bismillah Tergenerate Coy ###
def generate_images(num_images):
    # Noise
    noise = np.random.normal(0, 1, (num_images, latent_dim))

    # Generatornya
    generated_images = generator.predict(noise)

    generated_images = (generated_images + 1) / 2.0

    return generated_images

# 3. Generate beberapa gambar dan tampilkan
import matplotlib.pyplot as plt

def plot_generated_images(generated_images, num_rows=4, num_cols=4):
    plt.figure(figsize=(15, 15))

    for i in range(num_rows * num_cols):
        plt.subplot(num_rows, num_cols, i+1)
        plt.imshow(generated_images[i])
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Generate 16 gambar
images = generate_images(16)
plot_generated_images(images)

# 4. Simpan gambar yang dihasilkan
def save_generated_images(generated_images, output_dir='generated_images'):
    import os
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for i, image in enumerate(generated_images):
        # Ubah ke uint8 (0-255)
        image = (image * 255).astype(np.uint8)
        img_path = os.path.join(output_dir, f'generated_image_{i}.png')
        plt.imsave(img_path, image)

# Simpan gambar ke folder
save_generated_images(images)

# 5. Jika ingin menggunakan model generator yang sudah disimpan di lain waktu
from tensorflow.keras.models import load_model

# Load model
loaded_generator = load_model('generator_model.h5')

# Generate gambar baru menggunakan model yang di-load
def generate_new_images(num_images, generator_model):
    noise = np.random.normal(0, 1, (num_images, latent_dim))
    generated_images = generator_model.predict(noise)
    generated_images = (generated_images + 1) / 2.0
    return generated_images

# Contoh penggunaan
new_images = generate_new_images(4, loaded_generator)
# plot_generated_imagzes(new_images, 2, 2)

NameError: name 'generator' is not defined